In [51]:
import mlflow
import mlflow.sklearn
import azureml.contrib.mlflow
import azureml.core
from azureml.core import Workspace
import matplotlib.pyplot as plt

# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.33


In [52]:
from azureml.core import Workspace
subscription_id = "5763fde3-4253-480c-928f-dfe1e8888a57"
resource_group = "rsrcgrp_amlsdk"
workspace_name = "amlworkspaceaz"
workspace_region = "eastus2"

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded. Skip the workspace creation steps below")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

Workspace configuration succeeded. Skip the workspace creation steps below


In [33]:
ws = Workspace.from_config()

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [54]:
import mlflow
from mlflow.exceptions import MlflowException
from  mlflow.tracking import MlflowClient

#experimentPath = "/Users/" + username + "/AML_MlFlow_Experiment"
experimentPath = "AML_MlFlow_Experiment2"

try:
  experimentID = mlflow.create_experiment(experimentPath)
except MlflowException:
  experimentID = MlflowClient().get_experiment_by_name(experimentPath).experiment_id
  mlflow.set_experiment(experimentPath)

print("The experiment can be found at the path `{}` and has an experiment_id of `{}`".format(experimentPath, experimentID))

2019/05/20 14:21:12 ERROR mlflow.utils.rest_utils: API request to https://eastus2.experiments.azureml.net/history/v1.0/subscriptions/5763fde3-4253-480c-928f-dfe1e8888a57/resourceGroups/rsrcgrp_amlsdk/providers/Microsoft.MachineLearningServices/workspaces/amlworkspaceaz/api/2.0/preview/mlflow/experiments/create failed with code 500 != 200, retrying up to 2 more times. API response body: {
  "Error": {
    "Code": "ServiceError",
    "Message": "InternalServerError",
    "Target": null,
    "Details": [],
    "InnerError": null,
    "DebugInfo": {
      "Type": "Microsoft.MachineLearning.RunHistory.Services.MlFlow.Exceptions.ResourceAlreadyExistsException",
      "Message": "Experiment 'AML_MlFlow_Experiment2' already exists.",
      "StackTrace": "   at Microsoft.MachineLearning.RunHistory.Services.MlFlowService.CreateExperiment(Guid workspaceId, String experimentName) in /home/vsts/work/1/s/src/azureml-api/src/RunHistory/Services/MlFlowService.cs:line 94\n   at Microsoft.MachineLearnin

The experiment can be found at the path `AML_MlFlow_Experiment2` and has an experiment_id of `1fae32a0-dd68-487a-9a27-3d0427186e0c`


experiment_name = "AI_Airlft"
mlflow.set_experiment(experiment_name)

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("airbnb-cleaned-mlflow.csv")
X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

In [63]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

with mlflow.start_run(experiment_id=experimentID,run_name="Basic RF Experiment") as run:
  # Create model, train it, and create predictions
  rf = RandomForestRegressor()
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)
  
  # Log model
  mlflow.sklearn.log_model(rf, "random-forest-model")
  
  # Create metrics
  mse = mean_squared_error(y_test, predictions)
  r2 = r2_score(y_test, predictions)  
  print("  mse: {}".format(mse))
  print("  R2: {}".format(r2))

  runID = run.info.run_uuid
  experimentID = run.info.experiment_id
  print("Inside MLflow Run with run_id {} and experiment_id {}".format(runID, experimentID))
    
    # Log metrics
  mlflow.log_metric("mse", mse)
  


  mse: 6529.607485131691
  R2: 0.6811675600728937
Inside MLflow Run with run_id 94a489aa-4ef5-4d98-9444-d3c9ef31c31d and experiment_id 1fae32a0-dd68-487a-9a27-3d0427186e0c


In [64]:
ws.experiments[experimentPath]

Name,Workspace,Report Page,Docs Page
AML_MlFlow_Experiment2,amlworkspaceaz,Link to Azure Portal,Link to Documentation


In [70]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_path="random-forest-model", 
                                                      workspace=ws, 
                                                    
                                                      model_name="random-forest-model",
                                                      image_name="random-forest-model",
                                                      description="airbnb random-forest-model", 
                                                      tags={
                                                        "mse": str(mse),
                                                        "r2": str(r2),
                                                      },
                                                      synchronous=False)

Registering model random-forest-model


2019/05/20 16:28:34 INFO mlflow.azureml: Registered an Azure Model with name: `random-forest-model` and version: `4`


Creating image


2019/05/20 16:28:43 INFO mlflow.azureml: Building an Azure Container Image with name: `random-forest-model` and version: `4`


In [42]:
type(model_image)

azureml.core.image.container.ContainerImage

In [43]:
type(azure_model)

azureml.core.model.Model

In [41]:
model_image.wait_for_creation(show_output=True)

Running............
SucceededImage creation operation finished for image random-forest-model:1, operation "Succeeded"


In [44]:
from azureml.core.webservice import AciWebservice, Webservice

# Create the deployment 
dev_webservice_name = "airbnb-model"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, 
                                              image=model_image, 
                                              deployment_config=dev_webservice_deployment_config, 
                                              workspace=ws)

# Wait for the image deployment
dev_webservice.wait_for_deployment()

Creating service
ACI service creation operation finished, operation "Succeeded"


In [46]:
import numpy as np
import pandas as pd
from sklearn import datasets

data = pd.read_csv("airbnb-cleaned-mlflow.csv")
train, _ = train_test_split(data)
train_x = train.drop(["price"], axis=1)
sample = train_x.iloc[[0]]
query_input = list(sample.as_matrix().flatten())
sample_json = sample.to_json(orient="split")

In [47]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  print("Response: {}".format(response.text))
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [50]:
dev_scoring_uri = dev_webservice.scoring_uri
dev_scoring_uri

'http://52.191.221.120:80/score'

In [49]:
dev_prediction = query_endpoint_example(scoring_uri=dev_scoring_uri, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["host_total_listings_count","neighbourhood_cleansed","zipcode","latitude","longitude","property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type","minimum_nights","number_of_reviews","review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value"],"index":[3165],"data":[[1.0,15,12,37.7798763077,-122.4047214689,6,0,2.0,1.0,1.0,1.0,0,30.0,4.0,100.0,10.0,10.0,10.0,10.0,10.0,10.0]]}
Response: [306.9]
Received response: [306.9]


In [ ]:
dev_webservice.delete()

509  az login
510  az acr login --name amlworksacrtfhdqmhp
511  docker pull amlworksacrtfhdqmhp.azurecr.io/random-forest-model:1
512  docker images

514  docker run -it --name mlflow --rm amlworksacrtfhdqmhp.azurecr.io/random-forest-model:1 /bin/bash  
  
alexs-mbp-2:~ azeltov$ docker ps
CONTAINER ID        IMAGE                                                  COMMAND                 CREATED             STATUS              PORTS                NAMES
0eb033ecbca7        amlworksacrtfhdqmhp.azurecr.io/random-forest-model:1   "runsvdir /var/runit"   57 seconds ago      Up 57 seconds       5001/tcp, 8883/tcp   mlflow

516  docker run --name mlflow --rm -p 9998:5001 amlworksacrtfhdqmhp.azurecr.io/random-forest-model:1
 
docker stop mlflow

==========================================
Inside docker
==========================================
root@accde3169357:/var/azureml-app# ll

total 112
drwxr-xr-x 1 root root   4096 May 16 20:37 ./
drwxr-xr-x 1 root root   4096 Apr 24 19:32 ../
-rw-r--r-- 1 root root   8904 Apr 24 19:26 aml_blueprint.py
-rw-r--r-- 1 root root    498 Apr 24 19:26 aml_logger.py
-rw-r--r-- 1 root root  10593 Apr 24 19:26 app.py
drwxr-xr-x 2 root root   4096 Apr 24 19:26 appinsights/
drwxr-xr-x 3 root root   4096 Apr 24 19:26 azureml/
drwxr-xr-x 3 root root   4096 May 16 20:35 azureml-models/
-rw-r--r-- 1 root root    104 May 16 20:37 conda.yaml
-rw-r--r-- 1 root root    285 Apr 24 19:26 create_app.py
-rw-r--r-- 1  501 staff   520 May 16 20:35 execution_script.py
-rw-r--r-- 1 root root   1135 Apr 24 19:26 gunicorn_conf.py
-rw-r--r-- 1 root root    392 Apr 24 19:26 gunicorn_logging.conf
-rw-r--r-- 1 root root    884 Apr 24 19:26 kill_supervisor.py
-rw-r--r-- 1 root root   3220 May 16 20:35 main.py
-rwx------ 1 root root    455 May 16 20:35 model_config_map.json*
-rw-r--r-- 1 root root   4199 Apr 24 19:26 print_hook.py
-rw-r--r-- 1 root root     43 May 16 20:35 requirementsp_cq63m8.txt
-rw-r--r-- 1 root root    159 Apr 24 19:26 run_function_exception.py
-rw-r--r-- 1 root root   2991 Apr 24 19:26 swagger_template.json
-rw-r--r-- 1 root root    155 Apr 24 19:26 timeout_exception.py
-rw-r--r-- 1 root root     44 Apr 24 19:26 wsgi.py
-rw-r--r-- 1 root root    665 Apr 24 19:26 wsgi_request.py


alexs-mbp-2:kafka-openhack azeltov$ docker run --name mlflow --rm -p 9998:5001 amlworksacrtfhdqmhp.azurecr.io/random-forest-model:1
